Rapport : Quantitative Structure-Activity Relationship
Equipe : 

## **1 - Réprésentation des données**

In [255]:
# Importation des librairies
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer

#### **a) Analyse de l'ensemble des données**
#### Analyse des critères statistiques des attributs

In [256]:
# Lecture des données du xlsx
data_file = "QSAR_dataset.xlsx"
# Stockage des données dans un dataframe
data = pd.read_excel(data_file,index_col=0)
print(data.shape)

(154, 75)


In [257]:
# Statistiques descriptives des 74 attributs
data.describe()

,apol,ASA+,ASA-,a_count,a_donacc,a_heavy,a_hyd,a_IC,a_nC,a_nCl,...,VSA,vsa_acc,vsa_hyd,vsa_pol,vsurf_A,vsurf_R,vsurf_S,vsurf_V,Weight,zagreb
count,154.000000,152.000000,153.000000,154.000000,154.000000,153.000000,154.000000,153.000000,154.000000,154.000000,...,154.000000,154.000000,154.000000,154.000000,154.000000,1.470000e+02,141.000000,136.000000,154.000000,154.000000
mean,34.610698,105.781739,359.928668,23.909091,0.292208,18.875817,17.350649,33.912102,11.649351,3.110390,...,273.307303,8.076532,239.944812,9.086768,2.379611,-6.802721e+08,-66.497364,-2.501405,359.813016,101.350649
std,5.951534,62.391286,111.225998,4.895461,0.862625,5.596428,5.028718,9.714722,2.472152,2.954031,...,52.783753,14.721655,59.915749,15.129738,2.637952,8.247861e+09,73.647379,2.807324,132.955027,33.487395
min,17.148172,8.778115,122.917570,12.000000,0.000000,10.000000,6.000000,12.000000,6.000000,0.000000,...,140.102050,0.000000,67.651054,0.000000,0.011998,-1.000000e+11,-209.769584,-8.247237,128.174000,46.000000
25%,31.534723,70.909811,330.864750,22.000000,0.000000,17.000000,16.000000,30.541887,12.000000,0.000000,...,246.182178,0.000000,203.302167,0.000000,0.124434,2.696068e-01,-132.566487,-5.058933,291.992000,88.000000
50%,35.579689,98.659012,389.503510,22.000000,0.000000,18.000000,17.000000,31.868664,12.000000,4.000000,...,281.160615,0.000000,253.968020,0.000000,0.376156,8.136986e-01,-10.648449,-0.411358,360.881990,94.000000
75%,38.401845,139.629990,427.294460,25.750000,0.000000,19.000000,18.000000,37.087944,12.000000,6.000000,...,295.503230,13.566921,272.261230,13.566921,4.786711,9.972196e+00,-3.509363,-0.133136,410.317990,106.000000
max,52.422001,356.764860,622.904600,43.000000,4.000000,43.000000,40.000000,86.319427,20.000000,10.000000,...,432.120120,59.150364,475.687620,59.150364,7.429943,1.611555e+01,-0.338738,-0.013318,959.170960,246.000000



#### Prétraitement des données
**Traitement des données manquantes**
Nous avons décidé de procéder au traitement des données manquantes par imputation, au lieu de simplement supprimer les dites données. Les objets n'étant pas nombreux, il y a un risque d'obtenir des résultats faussés avec ce second choix.

In [258]:
# Détermination du type, du nombre et du pourcentage de valeurs manquantes par attribut
nb_m = data.isnull().sum().sort_values()
ratio_m = (data.isnull().sum()/data.shape[0]).sort_values()

In [259]:
manquant = pd.concat([nb_m, ratio_m], axis=1, sort=False)

In [260]:
# Affichage de ces données
df_manquants = pd.DataFrame({'Types': data[list(manquant.index.values)].dtypes,
                       'Nb manquants': nb_m,
                      '% de manquants': ratio_m,})
# On ne se concentre que sur les attributs aux valeurs manquantes
df_notnull = df_manquants[df_manquants["Nb manquants"]>0]
print(df_notnull)

           Types  Nb manquants  % de manquants
a_IC     float64             1        0.006494
a_heavy  float64             1        0.006494
ASA-     float64             1        0.006494
ASA+     float64             2        0.012987
vsurf_R  float64             7        0.045455
vsurf_S  float64            13        0.084416
vsurf_V  float64            18        0.116883


Tout d'abord, on observe que tous les attributs manquants sont numériques. De plus, on remarque que la proportion de données manquantes est différente pour chaque attribut, on n'est donc pas dans le cas du MMCA (Données manquantes de Manière Complètement Aléatoire)<sup>**1**</sup>. On considère que nos données sont dans le cas MA (Manquantes Aléatoirement), car c'est la situation la plus courante<sup>**2**</sup>. ceci nous dirige vers une imputation par régression, adaptée pour les données MA<sup>**3**</sup>.
Puisque la régression linéaire utilisée s'appui sur les données d'autres attributs numériques, nous avons décidé d'utiliser un dataframe sans l'attribut "Class", qui n'est pas un attribut numérique.

Afin de procéder à l'imputation des valeurs manquantes de ces attributs, il faut trouver ceux qui leur sont fortement corrélés.

In [261]:
# Liste des attributs avec des données manquantes
missing_attributes = ["a_IC","a_heavy","ASA-","ASA+","vsurf_R","vsurf_S","vsurf_V"]

# Création d'un Dataframe sans l'attribut "Class"
data_noclass = data.drop(columns = ["Class"])
df_corr = data_noclass.corr()

# Fonction permettant d'obtenir la liste des attributs fortement corrélés à attribut_name 
# On recherche au minimum un attribut fortement corrélé.

def high_correlation(attribute_name):
    coef = 0.9
    a = data_noclass.loc[:, data_noclass.corr()[attribute_name] > coef]
    # On ignore l'attribut étudié
    a = a.drop(columns=[attribute_name])
    while a.columns.empty :
        coef -= 0.1
        a = a = data_noclass.loc[:, data_noclass.corr()[attribute_name] > coef]
        a = a.drop(columns=[attribute_name])
    return list(a.columns)


In [262]:

for attribute in missing_attributes:
    random_data = pd.DataFrame(columns = [attribute])
    random_data[attribute] = data_noclass[attribute]
    predictor_attributes = high_correlation(attribute)

    data_noclass.fillna(data_noclass.select_dtypes(np.number).mean(), inplace=True)

    model = linear_model.LinearRegression()
    model.fit(X = data_noclass[predictor_attributes], y = data_noclass[attribute])
    
    #Standard Error of the regression estimates is equal to std() of the errors of each estimates
    predict = model.predict(data_noclass[predictor_attributes])
    std_error = (predict[data_noclass[attribute].notnull()] - data_noclass.loc[data_noclass[attribute].notnull(), attribute]).std()
    
    #observe that I preserve the index of the missing data from the original dataframe
    random_predict = np.random.normal(size = data_noclass[attribute].shape[0], 
                                      loc = predict, 
                                      scale = std_error)
    

    print(attribute)
    a = random_data[random_data[attribute].isna()]
    print(a)
    

a_IC
                                                    a_IC
SMILES                                                  
S(=O)(=O)([O-])C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)...   NaN
a_heavy
Empty DataFrame
Columns: [a_heavy]
Index: []
ASA-
Empty DataFrame
Columns: [ASA-]
Index: []
ASA+
Empty DataFrame
Columns: [ASA+]
Index: []
vsurf_R
Empty DataFrame
Columns: [vsurf_R]
Index: []
vsurf_S
Empty DataFrame
Columns: [vsurf_S]
Index: []
vsurf_V
Empty DataFrame
Columns: [vsurf_V]
Index: []


b)

c)

2 - Mesures de distance

a)

b)

3 - Choix du modèle de classification

b)

4 - Application

In [263]:
#okokoibn
def fun():
    return 897

### **Références**

1. https://stefvanbuuren.name/fimd/sec-MCAR.html
2. https://medium.com/analytics-vidhya/different-types-of-missing-data-59c87c046bf7
3. https://www.datacamp.com/tutorial/techniques-to-handle-missing-data-values